# Sentiment Analysis using NLTK using our dataset reviews from YELP

In [23]:
import os
import getpass
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pylab as plt
import json
import glob
import sys
from collections import Counter
from datetime import datetime as datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
import nltk
sys.path.append("..") # go to parent dir
import Code.basic_sentiment_analysis as sentiment_analysis
from tqdm import tqdm
%matplotlib inline

Path Locations

In [6]:
username = getpass.getuser()
if username == 'sdekel':
    text_data_path = 'C:\DataSets\yelp_dataset\dataset_restaurants'
    data_path = 'C:\DataSets\yelp_dataset\processed_data'
    dictionary_path = 'C:\\Users\\sdekel.TA7SHAYDE-L\\Dropbox\\PythonCourse\\Project\\Code\\dicts'
elif username == 'zahi':
    text_data_path ='D:\\users\\zahi\\OneDrive - Bar-Ilan University\\yelp_dataset_challenge_round9\\yelp2'
    data_path = 'D:\\users\\zahi\\OneDrive - Bar-Ilan University\\yelp_dataset_challenge_round9\\yelp2'
    dictionary_path = 'C:\\Users\\zahi\\Dropbox\\Project\\Code\\dicts'
else:
    data_path = ''  ## TODO: PUT YOUR DATA PATH HERE

Dataset Names

In [9]:
# text_data_name = 'yelp_academic_dataset_review_restaurants.json'
text_data_name = 'yelp_academic_dataset_review_restaurants.json'
text_data_fullpath = os.path.join( text_data_path, text_data_name )

business_data_name = 'yelp_academic_dataset_business_with_Clustered_location_with_Binary_Categories_andAttributes_with_checkins_clean_restaurants.csv'    
business_fullpath = os.path.join( data_path, business_data_name )

dict_dec_name = 'dec.yml'
dict_inc_name = 'inc.yml'
dict_inv_name = 'inv.yml'
dict_negative_name = 'negative.yml'
dict_positive_name = 'positive.yml'

dict_dec_name_fullpath = os.path.join( dictionary_path, dict_dec_name )
dict_inc_name_fullpath = os.path.join( dictionary_path, dict_inc_name )
dict_inv_name_fullpath = os.path.join( dictionary_path, dict_inv_name )
dict_negative_name_fullpath = os.path.join( dictionary_path, dict_negative_name )
dict_positive_name_fullpath = os.path.join( dictionary_path, dict_positive_name )


In [10]:
data_review   = pd.read_json(text_data_fullpath, lines=True )
business_data = pd.read_csv(business_fullpath, encoding='latin1')

In [11]:
data_review.head()

,business_id,cool,date,funny,review_id,stars,text,type,useful,user_id
0,4P-vTvE6cncJyUyLh73pxw,0,2014-03-29,0,6-hKBi-6RC3g7Mft0c-6qw,4,This place is a area staple! Been around for y...,review,0,PmgqNO0-5Y3e3UoR61TD7w
1,4P-vTvE6cncJyUyLh73pxw,0,2014-08-14,0,tRd0-mPa9O1TMJp_dw5khQ,4,Got my mojo back after having a few of their a...,review,0,kXUySHSlRgVrcR4Aa0HtGQ
2,4P-vTvE6cncJyUyLh73pxw,0,2012-10-20,0,8Mu56iQ-MYEyivqUVss0XA,4,"Don't go here for the decor, but the staff is ...",review,1,SYKPwRhnlKrW6yTvm7oPWg
3,4P-vTvE6cncJyUyLh73pxw,1,2012-08-25,0,ToC77cIEiMas9CPU7dt_fA,5,I believe in awarding stars bearing in mind th...,review,2,7LCG3o2KW2jgKgbKN0DQOg
4,4P-vTvE6cncJyUyLh73pxw,0,2015-07-27,0,6YC4o9yLc25DK8c6soOlaw,4,"If you like fried food and laid back, then thi...",review,0,iSdSNh1hjdE33LOwrFnFrg


In [12]:
business_data.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,business_id,hours_Friday,hours_Monday,hours_Saturday,hours_Sunday,hours_Thursday,hours_Tuesday,...,attribute_RestaurantsPriceRange2_2,attribute_RestaurantsPriceRange2_3,attribute_RestaurantsPriceRange2_4,attribute_RestaurantsAttire_casual,attribute_RestaurantsAttire_dressy,attribute_RestaurantsAttire_formal,attribute_NoiseLevel_quiet,attribute_NoiseLevel_loud,attribute_NoiseLevel_average,attribute_NoiseLevel_very_loud
0,0,0,0,EDqCEAGXVGCH4FJXgqtjqg,16.0,15.0,16.0,15.0,16.0,15.0,...,False,False,False,True,False,False,True,False,False,False
1,1,1,1,GDnbt3isfhd57T1QqU6flg,12.0,12.0,12.0,12.0,12.0,12.0,...,False,False,False,True,False,False,True,False,False,False
2,2,2,2,42romV8altAeuZuP2OC1gw,10.5,10.5,10.5,9.5,10.5,10.5,...,False,False,False,True,False,False,True,False,False,False
3,3,3,3,DNyYOxVAfu0oUcPNL1ljCQ,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,True,False,False,False,False,False,False
4,4,4,4,a1Ba6XeIOP48e64YFD0dMw,10.5,10.5,0.0,0.0,10.5,10.5,...,False,False,False,False,False,False,False,False,False,False


In [13]:
texts       = data_review['text']
business_id = business_data['business_id']

In [15]:
splitter = sentiment_analysis.Splitter()
postagger = sentiment_analysis.POSTagger()
dicttagger = sentiment_analysis.DictionaryTagger([ dict_positive_name_fullpath, dict_negative_name_fullpath,
                                                   dict_inc_name_fullpath     , dict_dec_name_fullpath      ,
                                                   dict_inv_name_fullpath ])                            

Check one exmaple

In [16]:
ind = 6
splitted_sentences = splitter.split(texts[ind])
pos_tagged_sentences = postagger.pos_tag(splitted_sentences)
dict_tagged_sentences = dicttagger.tag(pos_tagged_sentences)
score = sentiment_analysis.sentiment_score(dict_tagged_sentences)
print(texts[ind])
print(score)

I was not impressed with how gross this place is - I wasn't sure I wanted to order anything. The food was not outstanding. I think I would go across the street to the Tavern!!
-3.0


Run on the whole dataset

In [17]:
ind =3;
business_id_name =  business_id[ind]
# business_name = data_review['business_id'][0] 

print(business_id_name)
item_text = data_review[data_review['business_id'] == business_id_name ]

DNyYOxVAfu0oUcPNL1ljCQ


In [18]:
print(len(item_text.values))

7


In [19]:
# BusinessScore_df = pd.DataFrame( columns=['business_id', 'Score'] )
# BusinessScore_df = pd.DataFrame()

# BusinessScore_df = business_data[['business_id']]
# BusinessScore_df[['Score']] = []
sLength = len(business_id )
# business_data['Scores'] = pd.Series(np.zeros(sLength) )
business_id_name =  business_id[3]
BusinessScore_df = pd.DataFrame(  np.zeros((sLength,2)), columns=['business_id', 'Scores'])
# df3 = pd.DataFrame(np.random.randn(sLength, 2),columns=['a', 'b'])
BusinessScore_df['business_id'][3]= business_id_name
item_text = data_review[data_review['business_id'] == business_id_name ]
texts = item_text['text']
score = []
for text_item in texts:
     splitted_sentences = splitter.split(text_item)
     pos_tagged_sentences = postagger.pos_tag(splitted_sentences)
     dict_tagged_sentences = dicttagger.tag(pos_tagged_sentences)
     score.append(sentiment_analysis.sentiment_score(dict_tagged_sentences))
    
BusinessScore_df['Scores'][3] = str(score)
        
        

c:\users\zahi\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\zahi\appdata\local\programs\python\python35\lib\site-packages\pandas\core\indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [30]:
index = 0;
BusinessScore_df = pd.DataFrame(  np.zeros((sLength,2)), columns=['business_id', 'Scores'])
for business_id_name in tqdm(business_id):
    BusinessScore_df['business_id'][index] = business_id_name
    item_text = data_review[data_review['business_id'] == business_id_name ]
    texts = item_text['text']
    score = []
    for text_item in texts:
        splitted_sentences = splitter.split(text_item)
        pos_tagged_sentences = postagger.pos_tag(splitted_sentences)
        dict_tagged_sentences = dicttagger.tag(pos_tagged_sentences)
        score.append(sentiment_analysis.sentiment_score(dict_tagged_sentences))
        
    BusinessScore_df['Scores'][index] = str(score)
    index = index + 1

BusinessScore_df.to_csv("BusinessScore_df.csv")
    
        


  0%|                                                                                                    | 0/46417 [00:00<?, ?it/s]c:\users\zahi\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
c:\users\zahi\appdata\local\programs\python\python35\lib\site-packages\pandas\core\indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
100%|█████████████████████████████████████████████████████████████████████████████████████| 46417/46417 [38:26:11<00:00,  3.21s/it]


In [96]:
BusinessScore_df['Scores1']=BusinessScore_df['Scores'].apply(lambda x: x[1:-1].split(", "))
BusinessScore_df['Scores2']=BusinessScore_df['Scores1'].apply(lambda x:[float(item) for item in x] )
BusinessScore_df['ScoresAvg']=BusinessScore_df['Scores2'].apply(lambda x:sum(x)/len(x))
BusinessScore_df.head()

,business_id,Scores,Scores1,Scores2,ScoresAvg
0,EDqCEAGXVGCH4FJXgqtjqg,"[-1.0, 10.0, -1.0, 1.0, 0.0, 1.5, 3.0]","[-1.0, 10.0, -1.0, 1.0, 0.0, 1.5, 3.0]","[-1.0, 10.0, -1.0, 1.0, 0.0, 1.5, 3.0]",1.928571
1,GDnbt3isfhd57T1QqU6flg,"[10.0, 0.0, 0.0, -3.0, -5.0, 4.0, 4.0, 5.0, 6.0]","[10.0, 0.0, 0.0, -3.0, -5.0, 4.0, 4.0, 5.0, 6.0]","[10.0, 0.0, 0.0, -3.0, -5.0, 4.0, 4.0, 5.0, 6.0]",2.333333
2,42romV8altAeuZuP2OC1gw,"[1.0, 1.0, 5.0, -1.0, 3.0, 6.0, -3.0, 16.0, 5....","[1.0, 1.0, 5.0, -1.0, 3.0, 6.0, -3.0, 16.0, 5....","[1.0, 1.0, 5.0, -1.0, 3.0, 6.0, -3.0, 16.0, 5....",4.078947
3,DNyYOxVAfu0oUcPNL1ljCQ,"[1.0, 1.0, 2.0, 0.0, 1.0, -1.0, -1.0]","[1.0, 1.0, 2.0, 0.0, 1.0, -1.0, -1.0]","[1.0, 1.0, 2.0, 0.0, 1.0, -1.0, -1.0]",0.428571
4,a1Ba6XeIOP48e64YFD0dMw,"[5.0, 2.0, 9.0]","[5.0, 2.0, 9.0]","[5.0, 2.0, 9.0]",5.333333


In [89]:
business_data_and_scores=pd.concat([business_data,BusinessScore_df[['ScoresAvg']]], axis=1)

In [90]:
business_data_name_to_save = 'yelp_academic_dataset_business_with_Clustered_location_with_Binary_Categories_andAttributes_with_checkins_reviewsScore_clean_restaurants.csv'
business_fullpath = os.path.join( data_path, business_data_name_to_save )
business_data_and_scores.to_csv(business_fullpath )